# Capstone Project

# Libraries

In [1]:
# Import Library 

import re
import requests
from re import search

import unicodedata
from bs4 import BeautifulSoup
from bs4 import Tag

from bs4 import NavigableString
from bs4 import SoupStrainer
from pprint import pprint

import numpy as np
from numpy import array
import pandas as pd 

import csv
import pickle

from collections import Counter
import datetime

from IPython.display import display
from IPython.core import display as ICD
pd.set_option('display.max_columns', 100)


# Define Text Normalization Function

In [2]:
def restore_windows_1252_characters(restore_string):
    """
    Replace C1 control characters in the Unicode string s by the
    characters at the corresponding code points in Windows-1252,
    where possible.
    """

def to_windows_1252(match):
    try:
        return bytes([ord(match.group(0))]).decode('windows-1252')
    except UnicodeDecodeError:
        # No character at the corresponding code point: remove it.
        return ''

    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)

# Grab the Document Content

In [3]:
def grab_doc_content( brand, CIK ):
    
    company = {}
    
    company['auto'] = {}
    auto = {brand : CIK
           }
    company['auto'] = auto

    value = list(auto.copy().values())
    
    return value

In [4]:
GDC = grab_doc_content('Netflix', '0001065280')

In [5]:
master_filings_dict = {}
    
file_code = {}
    
file_text = {}

Accession_Number_URL = {}

In [6]:
# URL Directory For CIK    
dir_url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'
dir_url_list = [dir_url.format(x) for x in GDC]

#print('Directory URL: {}'.format(dir_url_list))
doc_url_list = [] 

# FOR-loop yielding Accession Numbers from CIK/URL Directory.
for CIK_num in GDC:

    doc_url = r'https://www.sec.gov/Archives/edgar/data/{CIKx}/{xx}/{yy}.txt'
    doc_url_new = doc_url.format(CIKx = CIK_num, xx='{xx}', yy ='{yy}')
    doc_url_list.append(doc_url_new)


In [7]:
def url_maker(dir_url):
    dir_url_list = [dir_url.format(x) for x in GDC]

    #print('Directory URL: {}'.format(dir_url_list))
    doc_url_list = [] 

    # FOR-loop yielding Accession Numbers from CIK/URL Directory.
    for CIK_num in GDC:

        doc_url = r'https://www.sec.gov/Archives/edgar/data/{CIKx}/{xx}/{yy}.txt'
        doc_url_new = doc_url.format(CIKx = CIK_num, xx='{xx}', yy ='{yy}')
        
        doc_url_list.append(doc_url_new)
        
        url_lists = zip( dir_url_list , doc_url_list )
        
    return url_lists

In [8]:
url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'
for dir_url, doc_url in url_maker(url): 
    print(1, dir_url)
    print(2, doc_url)

1 https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001065280&type=10-&dateb=&owner=include&count=100
2 https://www.sec.gov/Archives/edgar/data/0001065280/{xx}/{yy}.txt


In [9]:
Accession_Number_URL = {}
url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'

for dir_url, doc_url in url_maker(url):
    
    response = requests.get(dir_url)
    soup = BeautifulSoup(response.content, 'lxml')
    text = soup.get_text(strip=True)

    cleaned_text = re.findall('Acc-no: \d+-\d+-\d+' , text)

    accession_number = [n.replace('Acc-no: ', '') for n in cleaned_text]
    accessionnumber = [num.replace('-', '') for num in accession_number]

    accession_numbers = zip(accessionnumber, accession_number)
    
    cikk = [ cikk_.replace('CIK=', '') for cikk_ in re.findall('CIK=\d+', dir_url) ][0]
    CIKK = {cikk : accessionnumber}
    
    for (a,b) in accession_numbers: 
        master_filings_dict[b] = {}
        master_filings_dict[b]['sec_header_content'] = {}
        master_filings_dict[b]['filing_documents'] = None

        doc_url_single = doc_url.format(xx = a, yy = b)

        file_url_list = []

        

        file_url_list.append( doc_url_single )
        
        Accession_Number_URL.update({ b : file_url_list})

In [71]:
Accession_Number_URL

{'0001065280-20-000309': ['https://www.sec.gov/Archives/edgar/data/0001065280/000106528020000309/0001065280-20-000309.txt'],
 '0001065280-20-000155': ['https://www.sec.gov/Archives/edgar/data/0001065280/000106528020000155/0001065280-20-000155.txt'],
 '0001065280-20-000040': ['https://www.sec.gov/Archives/edgar/data/0001065280/000106528020000040/0001065280-20-000040.txt'],
 '0001065280-19-000368': ['https://www.sec.gov/Archives/edgar/data/0001065280/000106528019000368/0001065280-19-000368.txt'],
 '0001065280-19-000260': ['https://www.sec.gov/Archives/edgar/data/0001065280/000106528019000260/0001065280-19-000260.txt'],
 '0001065280-19-000157': ['https://www.sec.gov/Archives/edgar/data/0001065280/000106528019000157/0001065280-19-000157.txt'],
 '0001065280-19-000079': ['https://www.sec.gov/Archives/edgar/data/0001065280/000106528019000079/0001065280-19-000079.txt'],
 '0001065280-19-000043': ['https://www.sec.gov/Archives/edgar/data/0001065280/000106528019000043/0001065280-19-000043.txt'],


In [10]:
A_N = []
for AN, url in Accession_Number_URL.items():
    
    if AN == '0001193125-08-173275':
        break
    else:
        A_N.append(AN)
        print(url)
print(A_N, '\n')
print(len(A_N))
print(url)

['https://www.sec.gov/Archives/edgar/data/0001065280/000106528020000309/0001065280-20-000309.txt']
['https://www.sec.gov/Archives/edgar/data/0001065280/000106528020000155/0001065280-20-000155.txt']
['https://www.sec.gov/Archives/edgar/data/0001065280/000106528020000040/0001065280-20-000040.txt']
['https://www.sec.gov/Archives/edgar/data/0001065280/000106528019000368/0001065280-19-000368.txt']
['https://www.sec.gov/Archives/edgar/data/0001065280/000106528019000260/0001065280-19-000260.txt']
['https://www.sec.gov/Archives/edgar/data/0001065280/000106528019000157/0001065280-19-000157.txt']
['https://www.sec.gov/Archives/edgar/data/0001065280/000106528019000079/0001065280-19-000079.txt']
['https://www.sec.gov/Archives/edgar/data/0001065280/000106528019000043/0001065280-19-000043.txt']
['https://www.sec.gov/Archives/edgar/data/0001065280/000106528018000538/0001065280-18-000538.txt']
['https://www.sec.gov/Archives/edgar/data/0001065280/000106528018000360/0001065280-18-000360.txt']
['https://

## Save each Filing.

In [73]:
QorK = {'10-Q' : [],
        '10-K' : []
       }

for acc_num, url in Accession_Number_URL.items():
    
    master_document_dict = {}
    
    # create a stop point
    if acc_num == '0001193125-08-173275':
        break

    else:
        
        # grab the response
        response = requests.get(url[0])

    # Soupify
        # pass it through the parser, in this case let's just use lxml because the tags seem to follow xml.
        soup = BeautifulSoup(response.content, 'lxml')
        filing_document = soup.find('document')
        
    # Parsing
        # Document type ->> document_id
        document_id = filing_document.type.find(text=True, recursive=False).strip()
        # Document sequence ->> document_sequence
        document_sequence = filing_document.sequence.find(text=True, recursive=False).strip()
        # Document filename ->> document_filename
        document_filename = filing_document.filename.find(text=True, recursive=False).strip()
        # Document description ->> document_description
        document_description = filing_document.description.find(text=True, recursive=False).strip()
        
    # Storage    
        # initalize our document dictionary
        master_document_dict[document_id] = {}

        # add the different parts, we parsed up above.
        master_document_dict[document_id]['document_sequence'] = document_sequence
        master_document_dict[document_id]['document_filename'] = document_filename
        master_document_dict[document_id]['document_description'] = document_description
        
# Scraping
        # grab the text portion of the document, this will be used to split the document into pages.
        filing_doc_text = filing_document.find('text').extract()

        # find all the thematic breaks, these help define page numbers and page breaks.
        all_thematic_breaks = filing_doc_text.find_all('hr')

        # Locate and store page number via list comprehension.
        all_page_numbers = [thematic_break.previous_sibling.get_text(strip=True) for thematic_break in all_thematic_breaks if len(str(thematic_break))>6]

        # convert all thematic breaks to a string so it can be used for parsing
        all_thematic_breaks = [str(thematic_break) for thematic_break in all_thematic_breaks]

        # prep the document text for splitting, this means converting it to a string.
        filing_doc_string = str(filing_doc_text)
    
        # handle the case where there are thematic breaks.
        if len(all_thematic_breaks) > 0: 

            # define the regex delimiter pattern, this would just be all of our thematic breaks.
            regex_delimiter_pattern = '|'.join(map(re.escape, all_thematic_breaks))

            # split the document along each thematic break.
            split_filing_string = re.split(regex_delimiter_pattern, filing_doc_string)

            # store the document itself
            master_document_dict[document_id]['pages_code'] = split_filing_string

        # handle the case where there are no thematic breaks.
        elif len(all_thematic_breaks) == 0:

            # handles so it will display correctly.
            split_filing_string = all_thematic_breaks

            # store the document as is, since there are no thematic breaks. In other words, no splitting.
            master_document_dict[document_id]['pages_code'] = [filing_doc_string]
            
        # display some information to the user.
        print('-'*80)
        print('The document {} was parsed.'.format(document_id))
        print('There was {} page(s) found.'.format(len(all_page_numbers)))
        print('There was {} thematic breaks(s) found.'.format(len(all_thematic_breaks)))

        # store the documents in the master_filing_dictionary.
        master_filings_dict[acc_num]['filing_documents'] = master_document_dict
        
        # if document is 10-Q
        if document_id == '10-Q':
            QorK['10-Q'].append(acc_num) # add acc_num to QorK in 10-Q as key
        
        # if document is 10-K
        if document_id == '10-K':
            QorK['10-K'].append(acc_num) # add acc_num to QorK in 10-K as key
        
        del master_document_dict
        
        print('-'*80)
        print('All the documents for filing {} were parsed and stored.'.format(acc_num))
    


--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 36 page(s) found.
There was 37 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001065280-20-000309 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 30 page(s) found.
There was 31 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001065280-20-000155 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-K was parsed.
There was 69 page(s) found.
There was 69 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001065280-20-000040 were parsed and stored.
-

--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 35 page(s) found.
There was 37 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001065280-14-000028 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 34 page(s) found.
There was 36 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001065280-14-000021 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 30 page(s) found.
There was 32 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001065280-14-000012 were parsed and stored.
-

--------------------------------------------------------------------------------
The document 10-K was parsed.
There was 75 page(s) found.
There was 75 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001193125-09-037430 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 37 page(s) found.
There was 37 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001193125-08-222820 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 37 page(s) found.
There was 37 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001193125-08-173275 were parsed and stored.
-

AttributeError: 'NavigableString' object has no attribute 'get_text'

In [12]:
string_test = master_filings_dict['0000320193-20-000052']['filing_documents']['10-Q']['pages_code']


KeyError: '0000320193-20-000052'

## Anchor Check Function



In [12]:
# Variations of HTML anchors.
CBS_A = 'CONSOLIDATED BALANCE SHEETS'
CBS_B = 'Consolidated Balance Sheets'
CBS_C = 'Cash and cash equivalents'
# SoO_A = '<a name="Statements_of_Operations"' 
# SoO_B = 'id="CONSOLIDATED_STATEMENTS_OPERATIONS"'
# SoO3 = 'name="CONSOLIDATED_STATEMENTS_OPERATIONS"'
# CSoO10K = 'name="Consolidated_Statements_of_Operations"'
# CSoO10Knew = 'id="Consolidated_Statements_of_Operations"'

# CF_A = '<a name="Statements_of_Cash'
# CF_B = 'id="CONSOLIDATED_STATEMENTS_CASH_FLOWS"'
# CF3 = 'name="CONSOLIDATED_STATEMENTS_CASH_FLOWS"'
# CCF10K = 'name="Consolidated_Statements_of_Cash_Flows"'
# CCF10Knew = 'id="Consolidated_Statements_of_Cash_Flows"'

def anchor_check(page_code):
    
    # if the page has one of these anchors
    if (CBS_A in page_code) or (CBS_B in page_code) and (CBS_C in page_code):
        return True

    #elif (SoO_A in page_code) or (SoO_B in page_code) or (SoO3 in page_code) or (CSoO10K in page_code) or (CSoO10Knew in page_code) :
        #return True

    #elif (CF_A in page_code) or (CF_B in page_code) or (CF3 in page_code) or (CCF10K in page_code) or (CCF10Knew in page_code):
        #return True


    else:
        return False


## Cleaning Columns Function


In [13]:
month_dict = {'December' : 'Q4',
              'November' : 'Q4',
             'October' : 'Q4',
             'September' : 'Q3',
             'August': 'Q3',
             'July' : 'Q3',
             'June' : 'Q2',
             'May' : 'Q2',
             'April' : 'Q2',
             'March' : 'Q1',
             'Feburary' : 'Q1',
             'January' : 'Q1'}

In [14]:
col_name_variants = {
    'Inventor' : 'Inventory',
    'Total liabilities and' : 'Total liabilities and equity',
    'Common stock.*' : 'Common Stock',
    'Preferred stock.*': 'Preferred Stock',
    'Accounts receivable.*' : 'Accounts Receivable',
    'Accrued liabilities.*' : 'Accrued Liabilities',
    'Deferred revenue.*' : 'Deferred Revenue',
    'Redeemable noncontrolling interests in subsidiaries.*' : 'Noncontrolling interests in subsidiaries',
    'Short term investments.*' : 'Short term investments', 
    'Property|equipment.' : 'Property and Equipment',
    'Retained earnings.*' : 'Retained Earnings',
    'stockholders.*' : 'Shareholder Equity',
    '^Cash.*' : 'Cash',
    'Convertible senior notes.*' : 'Convertible Senior Notes'
}

In [15]:
def clean_col_name( name ):
    
    for detect, replacement in col_name_variants.items():

        if re.findall(detect, name, flags=re.I):

            n_column = name.replace(name, replacement) 

            return n_column 
            
    return name

In [16]:
# Throw column list-like object into cleaning function. 
def cleaning_column(r_column, m):
     
    column_list = []
    column_name = []
    
    for element_post, element in enumerate(r_column):
        
        # Handling the first "row" in the table i.e. column name.
        if element_post == 0:
#             print(element)
            ele_norm = unicodedata.normalize('NFKD', element)
            
            ele_norm = ele_norm.replace('net', '')
            
            ele_norm = ele_norm.replace( '-', ' ')
            
            s_ele_norm = re.sub(r"^\s+|\s+$", "", ele_norm)
            
            name = re.sub(r'(?=,|:|\.|\(){1}.*', repl='', string= s_ele_norm, flags=re.I)
            
            # clean_col_name function Choose old_if or new_dict
            clean_name = clean_col_name( name )
            ###
            column_name.append(clean_name)
            ###
            column_list.append(clean_name)
        
        # Handling the all the remaining "rows" in the table i.e. data/numbers.
        elif element not in ['\n$', '\n\xa0', '\n']:
                        
            n_element = element.replace( ',', '' )

            pattern = re.compile(r'[\d]+,[\d]+|[\d]+')

            res = pattern.findall(n_element)

            res = [int(ele) * m for ele in res]

            column_list.extend(res)

    return column_list
    

# Throw column list-like object into cleaning function. 
def cleaning_column(column, m):
     
    column_list = []

    for element_post, element in enumerate(column):
        
        # Handling the first "row" in the table i.e. column name.
        if element_post == 0:

            element = unicodedata.normalize('NFKD', element)
            
            element = element.replace( '\n', '')
            
            clean_ele = clean_col_name( element )
            
            column_list.append(clean_ele)
            
        # Use a dictionary to convert the Months into numbers.
        #elif element_post == :
            #month = re.sub('({})'.format('|'.join(map(re.escape, month_dict.keys()))), lambda m: month_dict[m.group()], dat)
            #column_list.extend(month)
        #elif element == '(' or ')': 
            #pint(element)
            #continue
            
        else:
            
            element = element.replace( '\n$' and '\n\xa0' and '\n', '0')
            
            element = element.replace( ',', '' )

            pattern = re.compile(r'[\d]+,[\d]+|[\d]+')

            res = pattern.findall(element)

            res = [int(ele) * m for ele in res]

            column_list.extend(res)

    return column_list
    

## Cleaning Tables Function

In [69]:
work_list = ['\xa0 \xa0', 'September\xa030, 2019', '\xa0', 'December\xa031, 2018']
test_list = ['\xa0', '\xa0', 'As of December\xa031,', '\xa0\xa0', '\xa0', '2019', '\xa0', '2018']
print(Quarter(work_list))
print()
print(Quarter(test_list))

column used for quarter ['\xa0 \xa0', 'September\xa030, 2019', '\xa0', 'December\xa031, 2018']
multi_list [['September', '30,', '2019'], ['December', '31,', '2018']]
new_column used for year ['September', '30,', '2019', 'December', '31,', '2018']
letter September
letter 30,
letter 2019
letter December
letter 31,
letter 2018
Q ['Q3', 'Q4']
(['Quarter', 'Q3'], ['Year', '2019'])

column used for quarter ['\xa0', '\xa0', 'As of December\xa031,', '\xa0\xa0', '\xa0', '2019', '\xa0', '2018']
multi_list [['As', 'of', 'December', '31,'], ['2019'], ['2018']]
new_column used for year ['As', 'of', 'December', '31,', '2019', '2018']
letter As
letter of
letter December
letter 31,
letter 2019
letter 2018
Q ['Q4']
(['Quarter', 'Q4'], ['Year', '2019'])


In [74]:
# New and imporved.
def Quarter( column ):
    
    print('column used for quarter', column)
    
    multi_list = [element.split() for element in column if element.split() != []]
    print('multi_list', multi_list)
    new_column = [item for sublist in multi_list for item in sublist]
    print('new_column used for year', new_column)
    months_key = list(month_dict.keys())
    Q = []
    Y = []
    
    for letter in new_column:
        letter = letter.encode('cp1252').decode("utf-8", "ignore")
#         print('letter', letter)
        try:
            month = re.findall( r'\w{}+'.format(months_key) , letter.replace(' ',  '') )
#             print('month', month)
            Q_num = month_dict[month[0]] 
    #         print('Q_num' , Q_num)
    
            if Q_num != []:
                Q.append(Q_num)
                
        except:
            continue
    #         print('~'*20)
        
    print('Q' , Q) 
#     Q_final = Q[0] 
#     print('Q_final' , Q_final)
    
    Q_final =['Quarter'] + Q[:1]
    
    
    for letter in new_column:   
        
        #print( letter.split(',') )
        
        Year = [re.compile(r'^[0-9]{4}$').findall(p_s_ele_norm) for p_s_ele_norm in letter.split(',') if re.compile(r'^[0-9]{4}$').findall(p_s_ele_norm) != []]
        Y.append(Year)
#     print(f'Y - {Y}')
    Y = [item for sublist in Y for item in sublist][0]
    
    Y.insert(0, 'Year')
    #Y.insert(1, 0)
    #Y.insert(1, 0)
    return Q_final, Y
    print( '\n', 'Q_final', Q_final, 'Y', Y)

# New and imporved. 
def Quarter( column ):
    
#     print(column)
    multi_list = [element.split() for element in column if element.split() != []]
#     print(multi_list)
    new_column = [item for sublist in multi_list for item in sublist]
#     print(new_column)
    Q = []
    Y = []
    
    for letter in column:
#         print('letter', letter)
        
        Q_num = [month_dict[word] for word in letter.split() if word in month_dict.keys() ]
#         print('Q_num' , Q_num)
        if Q_num != []:
            Q.append(Q_num)
#         print('~'*20)
        
#     print('Q' , Q) 
    Q_final = Q[0] 
#     print('Q_final' , Q_final)
    
    Q_final.insert(0, 'Quarter')
    
    for letter in new_column:        
        #print('new_column', new_column)
        Y = [re.compile(r'^[0-9]{4}$').findall(element) for element in new_column if re.compile(r'^[0-9]{4}$').findall(element) != []][0]
        
        Y.insert(0, 'Year')
        
    return Q_final, Y
#     print( '\n', 'Q_final', Q_final, 'Y', Y)

In [76]:
def table_extractor( page, multiplier, k ):
    
    # Convert string to BS object
    soup = BeautifulSoup(page, 'html5')

    # then get all the rows in the table.
    table_rows = soup.find_all('tr')
    
    if len( [tr.text for tr in table_rows[0].find_all('td')] ) < 3:
        del table_rows[0]
    
    single_table = []
    Date_column = []
    
    # Loop through each column, adding in three structures to single_table.
    for tr_post, tr in enumerate(table_rows):
        
        td = tr.find_all('td')

        column = [tr.text for tr in td]
        #print( 'tr_post', tr_post, len(column), column)
        # 10-Q 
        if k == '10-Q':
            
            if tr_post == 2:

                # Quarter Column
                col_Q, col_year = Quarter( column )
                single_table.append( col_Q )
                # Year Column
                single_table.append( col_year )

            elif tr_post > 3: 

                # Features Columns
                cleaned_data = cleaning_column( column , multiplier )
                if len(cleaned_data) > 10:
                    del cleaned_data[3]
                    
                single_table.append( cleaned_data )
                
        # 10-K
        elif k == '10-K':
            
            if tr_post == 1 or tr_post == 2:
                    
                Date_column.append(column)
                
            elif tr_post == 3:
                
                # Quarter Column
                col_date, col_year = Quarter([item for sublist in Date_column for item in sublist])
                single_table.append( col_date )
                # Year Column
                single_table.append( col_year )
                
            elif tr_post > 4:              
                
                cleaned_data = cleaning_column( column , 10**6 )
                if len(cleaned_data) > 10:
                    del cleaned_data[3]
                
                single_table.append( cleaned_data )
            
    return single_table

In [25]:
def df_clean( single_table ):
    #print(single_table)
    table_df = pd.DataFrame(single_table)

    table_df = table_df.transpose()  

    #table_df.drop(table_df.index[1:3], 0, inplace=True)

    #table_df.drop(table_df.index[2:], 0, inplace=True)

    table_df.columns = table_df.iloc[0]

    table_df = table_df.drop(table_df.index[0])
    
    return table_df

In [20]:
def table(page_code, k):
    
    doc_tables=[]

    for page_post, page in enumerate(page_code):
        
        if anchor_check(page) and ('in millions' in page):
            
            single_table = table_extractor( page, 10**6, k )
            
            table_df = df_clean(single_table)
                        
            doc_tables.append(table_df)
        
        elif anchor_check(page) and ('in thousands,' in page):
            
            #print('page_post', page_post)
            
            single_table = table_extractor( page, 10**3, k )

            table_df = df_clean(single_table)

            doc_tables.append(table_df)

                
    return doc_tables
    

# Exploring the Data

In [21]:
explore_dict = {}
for QK, AccNum_list in QorK.items():
    
    #for an_postition, acc_num in enumerate(AccNum_list): 
    if QK== '10-Q':
        Q_new = AccNum_list[0]
        Q_old = AccNum_list[-1]
        explore_dict[QK] = [Q_new, Q_old]

    if QK == '10-K':
        K_new = AccNum_list[0]
        K_old = AccNum_list[-1]
        explore_dict[QK] = [K_new, K_old]

In [22]:
explore_dict

{'10-Q': ['0001065280-20-000309', '0001065280-13-000020'],
 '10-K': ['0001065280-20-000040', '0001065280-13-000008']}

In [82]:
for QK, N_O in QorK.items():
    
    for an_postition, acc_num in enumerate(N_O):
        
        pages = master_filings_dict[acc_num]['filing_documents'][QK]['pages_code']
        print('\n', 'This is {} a {} document.'.format(acc_num, QK))
        test_df = table(pages, QK)
        table_df = test_df[0].drop([2]).reset_index(drop=True)
        print('Shape - ', table_df.shape)
        ICD.display(table_df)


 This is 0001065280-20-000309 a 10-Q document.
column used for quarter ['\xa0 \xa0', 'June\xa030, 2020', '\xa0', 'December\xa031, 2019']
multi_list [['June', '30,', '2020'], ['December', '31,', '2019']]
new_column used for year ['June', '30,', '2020', 'December', '31,', '2019']
Q ['Q2', 'Q4']
Shape -  (1, 30)


,Quarter,Year,Assets,Current assets,Cash,Other current assets,Total current assets,Content assets,Property and Equipment,Other non current assets,Total assets,Shareholder Equity,Current liabilities,Current content liabilities,Accounts payable,Accrued expenses and other liabilities,Deferred Revenue,Short term debt,Total current liabilities,Non current content liabilities,Long term debt,Other non current liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q2,2020,None,None,7153248000,1410891000,8564139000,25155117000,751941000,2704084000,37175281000,None,None,4664733000,446668000,986595000,1029261000,499161000,7626418000,3208164000,15294998000,1710948000,27840528000,None,None,3127813000,34072000,6241012000,9334753000,37175281000



 This is 0001065280-20-000155 a 10-Q document.
column used for quarter ['\xa0 \xa0', 'March\xa031, 2020', '\xa0', 'December\xa031, 2019']
multi_list [['March', '31,', '2020'], ['December', '31,', '2019']]
new_column used for year ['March', '31,', '2020', 'December', '31,', '2019']
Q ['Q1', 'Q4']
Shape -  (1, 30)


,Quarter,Year,Assets,Current assets,Cash,Other current assets,Total current assets,Content assets,Property and Equipment,Other non current assets,Total assets,Shareholder Equity,Current liabilities,Current content liabilities,Accounts payable,Accrued expenses and other liabilities,Deferred Revenue,Short term debt,Total current liabilities,Non current content liabilities,Long term debt,Other non current liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q1,2020,None,None,5151884000,1295897000,6447781000,25266889000,650455000,2694785000,35059910000,None,None,4761585000,545488000,1061090000,986753000,498809000,7853725000,3206051000,14170692000,1420148000,26650616000,None,None,2935532000,47054000,5520816000,8409294000,35059910000



 This is 0001065280-19-000368 a 10-Q document.
column used for quarter ['\xa0 \xa0', 'September\xa030, 2019', '\xa0', 'December\xa031, 2018']
multi_list [['September', '30,', '2019'], ['December', '31,', '2018']]
new_column used for year ['September', '30,', '2019', 'December', '31,', '2018']
Q ['Q3', 'Q4']
Shape -  (1, 30)


...

,Quarter,Year,Cash,Short term investments,Current content library,Prepaid content,Other current assets,,Total current assets,Content library,Property and Equipment,Deferred tax assets,Other non current assets,,Total assets,,Shareholder Equity,Current liabilities,Accounts payable,Accrued expenses,Current portion of lease financing obligations,Deferred Revenue,,Total current liabilities,Long term debt,Lease financing obligations,Other non current liabilities,,Total liabilities,Commitments and contingencies,Shareholder Equity,Preferred Stock,Common Stock,Additional paid in capital,Accumulated other comprehensive income,Retained Earnings,,Shareholder Equity,,Total liabilities and equity,
0,Q4,2010,194499000000,155888000000,181006000000,62217000000,47357000000,None,640967000000,180973000000,128570000000,17467000000,14090000000,None,982067000000,None,None,None,222824000000,36489000000,2083000000,127183000000,None,388579000000,200000000000,34123000000,69201000000,None,691903000000,None,None,None,53000000,51622000000,750000000,237739000000,None,290164000000,None,982067000000,None



 This is 0001193125-10-036181 a 10-K document.
column used for quarter ['\xa0', '\xa0\xa0', 'As of December\xa031,', '\xa0', '\xa0', '\xa0\xa0', '2009', '\xa0\xa0', '2008', '\xa0']
multi_list [['As', 'of', 'December', '31,'], ['2009'], ['2008']]
new_column used for year ['As', 'of', 'December', '31,', '2009', '2008']
Q ['Q4']
Shape -  (1, 43)


,Quarter,Year,Cash,Short term investments,Prepaid expenses,Prepaid revenue sharing expenses,Current content library,Other current assets,,Total current assets,Content library,Property and Equipment,Deferred tax assets,Other non current assets,,Total assets,,Shareholder Equity,Current liabilities,Accounts payable,Accrued expenses,Current portion of lease financing obligations,Deferred Revenue,,Total current liabilities,Long term debt,Lease financing obligations,Other non current liabilities,,Total liabilities,Commitments and contingencies,Shareholder Equity,Preferred Stock,Common Stock,Additional paid in capital,Treasury stock at cost,Accumulated other comprehensive income,Retained Earnings,,Shareholder Equity,,Total liabilities and equity,
0,Q4,2009,134224000000,186018000000,12491000000,17133000000,37329000000,23818000000,None,411013000000,108810000000,131653000000,15958000000,12300000000,None,679734000000,None,None,None,91475000000,33387000000,1410000000,100097000000,None,226369000000,200000000000,36572000000,17650000000,None,480591000000,None,None,None,53000000,338577000000,100020000000,273000000,198817000000,None,199143000000,None,679734000000,None



 This is 0001193125-09-037430 a 10-K document.
column used for quarter ['\xa0', '\xa0\xa0', 'As of\xa0December\xa031,', '\xa0', '\xa0\xa0', '2008', '\xa0', '\xa0', '2007']
multi_list [['As', 'of', 'December', '31,'], ['2008'], ['2007']]
new_column used for year ['As', 'of', 'December', '31,', '2008', '2007']
Q ['Q4']
Shape -  (1, 43)


,Quarter,Year,Cash,Short term investments,Prepaid expenses,Prepaid revenue sharing expenses,Current content library,Deferred tax assets,Other current assets,,Total current assets,Content library,Property and Equipment,Deferred tax assets,Other assets,,Total assets,,Shareholder Equity,Current liabilities,Accounts payable,Accrued expenses,Current portion of lease financing obligations,Deferred Revenue,,Total current liabilities,Lease financing obligations,Other liabilities,,Total liabilities,Commitments and contingencies,Shareholder Equity,Preferred Stock,Common Stock,Additional paid in capital,Treasury stock at cost,Accumulated other comprehensive income,Retained Earnings,,Shareholder Equity,,Total liabilities and equity,
0,Q4,2008,139881000000,157390000000,8122000000,18417000000,18691000000,5617000000,13329000000,None,361447000000,98547000000,124948000000,22409000000,10595000000,None,617946000000,None,None,None,100344000000,31394000000,1152000000,83127000000,None,216017000000,37988000000,16786000000,None,270791000000,None,None,None,62000000,338577000000,100020000000,84000000,108452000000,None,347155000000,None,617946000000,None


# Save Files Locally. 

In [83]:
Checklist = {}

for k, v in QorK.items():
    value_list = []
    for i, vv in enumerate(v):

        # first grab 10-Q documents only
        pages = master_filings_dict[v[i]]['filing_documents'][k]['pages_code']

        try:
            # Extract Table from html code and wrap it as a list of df.
            table_df_list = table(pages, k)      
            table_df = table_df_list[0].drop([2]).reset_index(drop=True)
            ICD.display(table_df)

            with open('{}-{}.csv'.format(v[i], k), "wb") as fp:   # Pickling
                pickle.dump(table_df, fp)

            # display a status to the user.
            print('All the pages from {} document {} have been tableized and saved.'.format(k, v[i]))
        
        except: 
            value_list.append(vv)
            print('Something went wrong in file {}, file type = {}'.format(v[i], k))
        
        print('-'*80) 
    Checklist['{}'.format(k)] = value_list
    del value_list

column used for quarter ['\xa0 \xa0', 'June\xa030, 2020', '\xa0', 'December\xa031, 2019']
multi_list [['June', '30,', '2020'], ['December', '31,', '2019']]
new_column used for year ['June', '30,', '2020', 'December', '31,', '2019']
Q ['Q2', 'Q4']


,Quarter,Year,Assets,Current assets,Cash,Other current assets,Total current assets,Content assets,Property and Equipment,Other non current assets,Total assets,Shareholder Equity,Current liabilities,Current content liabilities,Accounts payable,Accrued expenses and other liabilities,Deferred Revenue,Short term debt,Total current liabilities,Non current content liabilities,Long term debt,Other non current liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q2,2020,None,None,7153248000,1410891000,8564139000,25155117000,751941000,2704084000,37175281000,None,None,4664733000,446668000,986595000,1029261000,499161000,7626418000,3208164000,15294998000,1710948000,27840528000,None,None,3127813000,34072000,6241012000,9334753000,37175281000


All the pages from 10-Q document 0001065280-20-000309 have been tableized and saved.
--------------------------------------------------------------------------------
column used for quarter ['\xa0 \xa0', 'March\xa031, 2020', '\xa0', 'December\xa031, 2019']
multi_list [['March', '31,', '2020'], ['December', '31,', '2019']]
new_column used for year ['March', '31,', '2020', 'December', '31,', '2019']
Q ['Q1', 'Q4']


,Quarter,Year,Assets,Current assets,Cash,Other current assets,Total current assets,Content assets,Property and Equipment,Other non current assets,Total assets,Shareholder Equity,Current liabilities,Current content liabilities,Accounts payable,Accrued expenses and other liabilities,Deferred Revenue,Short term debt,Total current liabilities,Non current content liabilities,Long term debt,Other non current liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q1,2020,None,None,5151884000,1295897000,6447781000,25266889000,650455000,2694785000,35059910000,None,None,4761585000,545488000,1061090000,986753000,498809000,7853725000,3206051000,14170692000,1420148000,26650616000,None,None,2935532000,47054000,5520816000,8409294000,35059910000


All the pages from 10-Q document 0001065280-20-000155 have been tableized and saved.
--------------------------------------------------------------------------------
column used for quarter ['\xa0 \xa0', 'September\xa030, 2019', '\xa0', 'December\xa031, 2018']
multi_list [['September', '30,', '2019'], ['December', '31,', '2018']]
new_column used for year ['September', '30,', '2019', 'December', '31,', '2018']
Q ['Q3', 'Q4']


...

All the pages from 10-K document 0001193125-11-040217 have been tableized and saved.
--------------------------------------------------------------------------------
column used for quarter ['\xa0', '\xa0\xa0', 'As of December\xa031,', '\xa0', '\xa0', '\xa0\xa0', '2009', '\xa0\xa0', '2008', '\xa0']
multi_list [['As', 'of', 'December', '31,'], ['2009'], ['2008']]
new_column used for year ['As', 'of', 'December', '31,', '2009', '2008']
Q ['Q4']


,Quarter,Year,Cash,Short term investments,Prepaid expenses,Prepaid revenue sharing expenses,Current content library,Other current assets,,Total current assets,Content library,Property and Equipment,Deferred tax assets,Other non current assets,,Total assets,,Shareholder Equity,Current liabilities,Accounts payable,Accrued expenses,Current portion of lease financing obligations,Deferred Revenue,,Total current liabilities,Long term debt,Lease financing obligations,Other non current liabilities,,Total liabilities,Commitments and contingencies,Shareholder Equity,Preferred Stock,Common Stock,Additional paid in capital,Treasury stock at cost,Accumulated other comprehensive income,Retained Earnings,,Shareholder Equity,,Total liabilities and equity,
0,Q4,2009,134224000000,186018000000,12491000000,17133000000,37329000000,23818000000,None,411013000000,108810000000,131653000000,15958000000,12300000000,None,679734000000,None,None,None,91475000000,33387000000,1410000000,100097000000,None,226369000000,200000000000,36572000000,17650000000,None,480591000000,None,None,None,53000000,338577000000,100020000000,273000000,198817000000,None,199143000000,None,679734000000,None


All the pages from 10-K document 0001193125-10-036181 have been tableized and saved.
--------------------------------------------------------------------------------
column used for quarter ['\xa0', '\xa0\xa0', 'As of\xa0December\xa031,', '\xa0', '\xa0\xa0', '2008', '\xa0', '\xa0', '2007']
multi_list [['As', 'of', 'December', '31,'], ['2008'], ['2007']]
new_column used for year ['As', 'of', 'December', '31,', '2008', '2007']
Q ['Q4']


,Quarter,Year,Cash,Short term investments,Prepaid expenses,Prepaid revenue sharing expenses,Current content library,Deferred tax assets,Other current assets,,Total current assets,Content library,Property and Equipment,Deferred tax assets,Other assets,,Total assets,,Shareholder Equity,Current liabilities,Accounts payable,Accrued expenses,Current portion of lease financing obligations,Deferred Revenue,,Total current liabilities,Lease financing obligations,Other liabilities,,Total liabilities,Commitments and contingencies,Shareholder Equity,Preferred Stock,Common Stock,Additional paid in capital,Treasury stock at cost,Accumulated other comprehensive income,Retained Earnings,,Shareholder Equity,,Total liabilities and equity,
0,Q4,2008,139881000000,157390000000,8122000000,18417000000,18691000000,5617000000,13329000000,None,361447000000,98547000000,124948000000,22409000000,10595000000,None,617946000000,None,None,None,100344000000,31394000000,1152000000,83127000000,None,216017000000,37988000000,16786000000,None,270791000000,None,None,None,62000000,338577000000,100020000000,84000000,108452000000,None,347155000000,None,617946000000,None


All the pages from 10-K document 0001193125-09-037430 have been tableized and saved.
--------------------------------------------------------------------------------


In [84]:
# Checklist of erroneous filings.  
for a, b in Checklist.items():
    if b != []:
        print('These have a problem.', '\n', a, b)

    else: 
        print(a, 'has no problem!! :) ' , '\n')
    
print(Checklist)

10-Q has no problem!! :)  

10-K has no problem!! :)  

{'10-Q': [], '10-K': []}


In [80]:
# Bring QorK over to Part 2. 
QorK = {'10-Q': ['0001065280-20-000309',
  '0001065280-20-000155',
  '0001065280-19-000368',
  '0001065280-19-000260',
  '0001065280-19-000157',
  '0001065280-18-000538',
  '0001065280-18-000360',
  '0001065280-18-000205',
  '0001065280-17-000180',
  '0001065280-17-000046',
  '0001628280-17-003939',
  '0001065280-16-000081',
  '0001065280-16-000072',
  '0001065280-16-000057',
  '0001065280-15-000038',
  '0001065280-15-000031',
  '0001065280-15-000017',
  '0001065280-14-000028',
  '0001065280-14-000021',
  '0001065280-14-000012',
  '0001065280-13-000036',
  '0001065280-13-000030',
  '0001065280-13-000020',
  '0001065280-12-000020',
  '0001065280-12-000012',
  '0001445305-12-001242',
  '0001193125-11-284366',
  '0001193125-11-198669',
  '0001193125-11-112061',
  '0001193125-10-235785',
  '0001193125-10-167382',
  '0001193125-10-095215',
  '0001193125-09-213071',
  '0001193125-09-160729',
  '0001193125-09-103685',
  '0001193125-08-222820'],
 '10-K': ['0001065280-20-000040',
  '0001065280-19-000043',
  '0001065280-18-000069',
  '0001628280-17-000496',
  '0001065280-16-000047',
  '0001065280-15-000006',
  '0001065280-14-000006',
  '0001065280-13-000008',
  '0001193125-12-053009',
  '0001193125-11-040217',
  '0001193125-10-036181',
  '0001193125-09-037430']}